In [1]:
#파이토치
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#판다스
import pandas as pd

#numpy
import numpy as np

## 데이터 로드

In [2]:
train = pd.read_csv('data/titanic/train.csv')
test = pd.read_csv('data/titanic/test.csv')

## 데이터 전처리

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#'PassengerId','Name','Ticket','Cabin' 은 어려우니 그냥 없애주자
drop_list=['PassengerId','Name','Ticket','Cabin','Fare','Embarked']

ID = test

train = train.drop(drop_list, axis=1)
test = test.drop(drop_list, axis=1)

In [5]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


In [6]:
# Sex 는 0과 1로 나타내기 : Female = 1, male = 0

combine = [train, test]

for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,0,22.0,1,0
1,1,1,1,38.0,1,0
2,1,3,1,26.0,0,0
3,1,1,1,35.0,1,0
4,0,3,0,35.0,0,0


## null 데이터 있는지 확인하고 처리하기

In [7]:
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
dtype: int64

In [8]:
test.isnull().sum()

Pclass     0
Sex        0
Age       86
SibSp      0
Parch      0
dtype: int64

In [9]:
#Age에 빈값은 mean값을 넣음
test['Age'].fillna(test['Age'].dropna().mean(), inplace=True)
train['Age'].fillna(train['Age'].dropna().mean(), inplace=True)

train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
dtype: int64

## 데이터 나누기

In [10]:
train_Y = train['Survived']
train_X = train.drop('Survived', axis=1)

train_Y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [11]:
train_X.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,0,22.0,1,0
1,1,1,38.0,1,0
2,3,1,26.0,0,0
3,1,1,35.0,1,0
4,3,0,35.0,0,0


## 텐서생성

In [12]:
train_X = torch.from_numpy(train_X.as_matrix()).float()
train_Y = torch.from_numpy(train_Y.as_matrix()).long()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [25]:
test_X = torch.from_numpy(test.as_matrix()).float()
test.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(418, 5)

In [13]:
print(train_X.shape)
print(train_Y.shape)

torch.Size([891, 5])
torch.Size([891])


In [14]:
train = TensorDataset( train_X, train_Y)

In [15]:
#미니 배치로 분활
train_loader = DataLoader(train, batch_size=100, shuffle=True)

## 모델 만들기

In [16]:
class Net( nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(5, 96)
    self.fc2 = nn.Linear(96, 96)
    self.fc3 = nn.Linear(96, 96)
    self.fc4 = nn.Linear(96, 96)
    self.fc5 = nn.Linear(96, 96)
    self.fc6 = nn.Linear(96, 2)
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))
    x = self.fc6(x)
    return F.log_softmax(x)

In [17]:
model = Net()

In [18]:
#오차함수 객체
criterion = nn.CrossEntropyLoss()

In [21]:
#최적화를 담당할 객체
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [26]:
#학습 시작
for epoch in range(500):
  total_loss = 0
  
  #분할 해둔 데이터를 꺼내옴
  for train_x, train_y in train_loader:
    #계산 그래프 구성
    train_x, train_y = Variable(train_x), Variable(train_y)
    
    #경사 초기화
    optimizer.zero_grad()
    
    #순전파 계산
    output = model(train_x)
    
    #오차계산
    loss = criterion(output, train_y)
    
    #역전파
    loss.backward()
    
    #가중치 업데이트
    optimizer.step()
    
    #누적 오차 계산
    total_loss += loss.item()
    
  if (epoch + 1) % 50 == 0:
    print(epoch+1, total_loss)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


50 3.308384746313095
100 3.353565439581871
150 3.318642482161522
200 3.161846548318863
250 3.169611543416977
300 3.0972580164670944
350 3.159860759973526
400 3.1668108999729156
450 3.218679204583168
500 2.9866019636392593


## 예측

In [27]:
test_x = Variable(test_X)

In [31]:
result = torch.max(model(test_x).data, 1)[1]
result = result.numpy()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [33]:
submission = pd.DataFrame({"PassengerId":ID["PassengerId"],"Survived" : result})
submission.to_csv('submission.csv', index=False)